In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
cwd = os.path.join(os.getcwd(), 'Group Coursework Brief-20221106', 'Data_Files', 'Data_Files')
dirName_trainData = os.path.join(cwd, 'epl-training.csv')

In [3]:
df_epl_train = pd.read_csv(dirName_trainData)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/neilbadal/Desktop/COMP0036 CW/Group Coursework Brief-20221106/Data_Files/Data_Files/epl-training.csv'

In [ ]:
df_epl_train.head()

In [ ]:
# Transform the date column from strings into datetime objects
df_epl_train["Date"] = pd.to_datetime(df_epl_train["Date"], dayfirst=True)

In [ ]:
df_epl_train.head()

AIM: A function that takes as input the date, HomeTeam and AwayTeam. It will filter the df_epl_train dataframe for matches between HomeTeam and AwayTeam that took place before the input date. Then take an average of the 
columns like HR, AR, etc. This will provide us with the past stats for games played in past between the two teams. We can then use these past stats (between the two teams) as features to input into the classifier.

First, filter the dataframe to include only matches where date is less than date specified and also only include matches where HomeTeam=input(HomeTeam) and AwayTeam=input(AwayTeam):

In [ ]:
# This function will take as input a date, HomeTeam and AwayTeam and output a filtered dataframe where the matches shown are played before input data and match is between HomeTeam and AwayTeam

# For Example:
# date = "24/06/2020"
# HomeTeam = "Newcastle"
# AwayTeam = "Aston Villa"

def filter_dataframe(date, HomeTeam, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && HomeTeam=input(HomeTeam) && AwayTeam=input(AwayTeam)
    df_epl_train_filtered = df_epl_train.copy()
    df_epl_train_filtered = df_epl_train_filtered[(df_epl_train.Date<date) & (df_epl_train.HomeTeam==HomeTeam) & (df_epl_train.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_epl_train_filtered

# An example to see what the function does:
print(filter_dataframe("24/06/2020", "Newcastle", "Aston Villa"))

We now find the average of each of the columns that we need from this filtered dataframe e.g. HST, AST:

In [ ]:
# This function takes as input the filtered dataframe from previous cell, features to average and a dictionary,
# it then appends an average of each feature to the dictionary

def average_columns(features, avg_features, filtered_df):
    for feature in features:
        df_col_means = df_epl_train_filtered[feature].mean()
        avg_features[feature].append(df_col_means)

We now run the two functions on each row of the original dataframe to fill the dictionary with averages for each row

In [ ]:
# Run the two functions for each row of our df_epl_train dataframe to fill dictionary with AVG for each match
# NOTE: Some matches won't have past stats since the two teams may not have played against each other in past or we might not have the data

# These are the features we want to get averages for
features = ["FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]
avg_features = {
                    "FTHG": [],
                    "FTAG": [],
                    "HTHG": [],
                    "HTAG": [],
                    "HS"  : [],
                    "AS"  : [],
                    "HST" : [],
                    "AST" : [],
                    "HF"  : [],
                    "AF"  : [],
                    "HC"  : [],
                    "AC"  : [],
                    "HY"  : [],
                    "AY"  : [],
                    "HR"  : [],
                    "AR"  : []
                }

# Run the two functions on each row of the df_epl_train and fill the dictionary
# For each row in the dataframe
for index, row in df_epl_train.iterrows():
    # Filter the dataframe to only show matches played between those teams and before the certain date
    df_epl_train_filtered = filter_dataframe(row["Date"],row["HomeTeam"],row["AwayTeam"])
    # Get averages from the filtered dataframe and add the the dictionary
    average_columns(features, avg_features, df_epl_train_filtered)

Add these average feature lists in the dictionary back into the original dataframe as columns:

In [ ]:
# Add a column for each of these feature averages using our list of values from the dictionary
df_epl_train_updated = df_epl_train.copy()
features = ["FTHG","FTAG","HTHG","HTAG","HS","AS","HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]

for feature in features:
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals

This is the new dataframe with the added columns with past average statistics for each row:

In [ ]:
# Now this dataframe contains our original data + the average of the past stats for each row
df_epl_train_updated

NOTE: Some rows have nan values, these are matches where teams might not have played each other before (or we dont have the past match data for them)

So we need to remove these rows (from the dataframe) where there are nan values. This is required to run the classifier training

In [ ]:
# NOTE: We must remove the rows in the dataframe where the average values of stats/features are 'nan';
# we get these values because either the two teams have not played a match in the past OR because we have
# not got the past stats for these matches. We cannot use the 'nan' values for the classifier training and 
# hence have to remove these rows. We can then train a classifier using this final dataframe.

# In the final model/classifier, in the case where we DO NOT have these past stats of the two teams playing, 
# we need to switch back to using the OLD classifier which only took the 4 basic fetaures: day, month, 
# HomeTeam and AwayTeam.

# In the case where we DO have these past stats for two teams playing each other, we can use this model/classifier 
# and input the features like HST_AVG and AST_AVG. We would find these by using the filter_dataframe() and 
# average_columns() functions to find them for any two specific teams playing each other on some date.

# Remove any rows with nan
df_epl_train_final = df_epl_train_updated.dropna()
df_epl_train_final

In [ ]:
# Turn the catergorical data into labels using same method from before
df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes

# Transform the date column into day and month columns and Add into dataframe (Extract days & months from date)
df_epl_train_final["Date"] = pd.to_datetime(df_epl_train_final["Date"])
df_epl_train_final["Day"] = df_epl_train_final["Date"].dt.day
df_epl_train_final["Month"] = df_epl_train_final["Date"].dt.month 
df_epl_train_final["Year"] = df_epl_train_final["Date"].dt.year

# Check the final updated dataframe
df_epl_train_final

Create our X and y matrix and split into a training and test set:

In [ ]:
# Create the input features matrix X (made of day, month, HomeTeam, AwayTeam, FTHG_AVG, FTAG_AVG, etc)
# Create the output values y vector (made of FTR)
# Take these values from the transformed dataframe

X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc','FTHG_AVG','FTAG_AVG','HTHG_AVG','HTAG_AVG','HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
y = df_epl_train_final.loc[:,'FTR'].values

# Split the training data in a 80-20 split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=22)

# Encode the y output values as well
FTR_encoder = LabelEncoder()
y_train = FTR_encoder.fit_transform(y_train);

Now we can test using the different classifiers:

In [ ]:
# Create an empty Tree model
DT_Model = DecisionTreeClassifier(random_state=42)
# Fit the model using training data
DT_Model.fit(X_train, y_train)
# Make predictions using the model we have created
DT_predictions_test = DT_Model.predict(X_test)
# Reconverting prediction values (i.e. 0, 1 or 2) back into (H, D or A) using the FTR_encoder defined in earlier cell
DT_predictions_test = FTR_encoder.inverse_transform(DT_predictions_test)

In [ ]:
print(accuracy_score(DT_predictions_test, y_test))
print(classification_report(DT_predictions_test, y_test))

In [ ]:
# Create an empty KNN model
KNN_Model = KNeighborsClassifier(n_neighbors=6)
# Fit the model using training data
KNN_Model.fit(X_train, y_train)
# Make predictions using the model we have created
KNN_predictions_test = KNN_Model.predict(X_test)
KNN_predictions_test = FTR_encoder.inverse_transform(KNN_predictions_test)

In [ ]:
print(accuracy_score(KNN_predictions_test, y_test))
print(classification_report(KNN_predictions_test, y_test))

In [ ]:
# Create an empty Random Forest model
RF_Model = RandomForestClassifier(n_estimators=50, random_state=42)
# Fit the model using training data
RF_Model.fit(X_train, y_train)
# Make predictions using the model we have created
RF_predictions_test = RF_Model.predict(X_test)
RF_predictions_test = FTR_encoder.inverse_transform(RF_predictions_test)

In [ ]:
print(accuracy_score(RF_predictions_test, y_test))
print(classification_report(RF_predictions_test, y_test))